# ViT Segmentation

Como foi explicado no primeiro notebook sobre ViT, os Transformers são modelos que podem ser entendidos como *Encoders*. Isso significa que podemos utilizar o ViT apenas para a extração de *features* do nosso problema e, em seguida, adicionar uma *head* para realizar a tarefa principal, que pode ser classificação, segmentação ou alguma abordagem *multi-modal*. Dessa forma, neste notebook, iremos explorar como usar um ViT pré-treinado, construir uma *head* de segmentação e aplicá-lo à tarefa de segmentação do *Rock Dataset*.


## Configuração

Importando módulos necessários

In [ ]:
%load_ext nbproxy

## Imports

In [ ]:
# Bibliotecas principais
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

# Transformações e manipulação de dados
import torchvision
from torchvision import transforms, models, datasets

# Utilitários de visualização e manipulação de dados
import os
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from skimage import io, transform
from sklearn import metrics
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import ViTModel

# Divisão de dados
from sklearn.model_selection import train_test_split



## Funções Auxiliares

Funções para avaliação de métricas, apresentação de imagens e das predições

In [ ]:
def evaluate(preds, labels):
    f1_list = []
    iou_list = []

    for i in tqdm(range(len(preds)), desc='Metrics'):
        f1 = metrics.f1_score(labels[i].flatten(), preds[i].flatten())
        iou = metrics.jaccard_score(labels[i].flatten(), preds[i].flatten())

        f1_list.append(f1)
        iou_list.append(iou)

    f1_list = np.asarray(f1_list)
    iou_list = np.asarray(iou_list)

    return f1_list, iou_list

def show_image(img, mask):
    img = img.cpu().numpy().transpose(1, 2, 0)  # (C, H, W) -> (H, W, C)
    img = np.clip(img, 0, 1) 
    
    if mask.ndim == 3:
        mask = mask.squeeze(0)  # (1, H, W) -> (H, W)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    axes[0].imshow(img)
    axes[0].axis('off') 
    axes[0].set_title("Imagem")

    axes[1].imshow(mask, cmap='gray')
    axes[1].axis('off')  
    axes[1].set_title("Máscara")

    plt.show()

def show_image_and_mask_prediction(img, mask, pred):
    img = img.cpu().numpy().transpose(1, 2, 0)  # (C, H, W) -> (H, W, C)
    img = np.clip(img, 0, 1)  

    if mask.ndim == 3:  
        mask = mask.squeeze(0)  # (1, H, W) -> (H, W)
        
    pred = torch.argmax(pred, dim=0).cpu().numpy() 
    
    if pred.ndim == 3: 
        pred = pred.squeeze(0)  # (1, H, W) -> (H, W)

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    axes[0].imshow(img)
    axes[0].axis('off')  
    axes[0].set_title("Imagem")

    axes[1].imshow(mask, cmap='gray')
    axes[1].axis('off')  
    axes[1].set_title("Máscara")

    axes[2].imshow(pred, cmap='gray')
    axes[2].axis('off')  
    axes[2].set_title("Previsão")

    plt.show()


## Dataset - RockDataset

## Carregamento do Dataset `Rockdataset`

Aqui será feito o carregamento do dataset `Rockdataset`, que já foi utilizado em uma tarefa anterior de segmentação com uma rede convolucional U-Net, na semana de CNN. 

O carregamento de dados será diferente desta vez, pois utilizaremos uma versão pré-pronta e pré-treinada do Vision Transformer para essa tarefa. Essa abordagem exige que a imagem e a máscara estejam em um formato específico:

- Imagem: `(batch_size, 3 , altura, largura)` → 3 canais (RGB), 224x224
- Máscara: `(batch_size, 1, altura, largura)` → 1 canal (grayscal, 224x224

In [ ]:
class RockDataset(Dataset):
    def __init__(
            self,
            is_train: bool,
            crop_size: int = 256,
            num_classes: int = 2,
            datapath: str = '/pgeoprj2/ciag2024/dados/drp-benchmarks/images/grosmont',
            transform = None
        ):
        self.is_train = is_train
        self.crop_size = crop_size
        self.num_classes = num_classes
 
        self.img_path = os.path.join(datapath, 'tif')
        self.mask_path = os.path.join(datapath, 'segmented-kongju.raw')
 
        self.image_filenames, self.mask_vol = self.make_dataset()
 
        self.transform = transform
 
        if len(self.image_filenames) == 0:
            raise RuntimeError('Found 0 images, please check the data set.')
 
    def make_dataset(self):
        if self.is_train:
            mask_vol = np.fromfile(open(self.mask_path, 'rb'), dtype=np.int8).reshape(1024, 1024, 1024)
            mask_vol = mask_vol.transpose((0, 2, 1))[:-64]
        else:
            mask_vol = np.fromfile(open(self.mask_path, 'rb'), dtype=np.int8).reshape(1024, 1024, 1024)
            mask_vol = mask_vol.transpose((0, 2, 1))[-64:]
 
        if self.is_train:
            image_filenames = sorted([f for f in os.listdir(self.img_path) if os.path.isfile(os.path.join(self.img_path, f))])[:-64]
        else:
            image_filenames = sorted([f for f in os.listdir(self.img_path) if os.path.isfile(os.path.join(self.img_path, f))])[-64:]
 
        return image_filenames, mask_vol
 
    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        filename = self.image_filenames[idx]
        img_path = os.path.join(self.img_path, filename)
    
        img = io.imread(img_path)
        mask = self.mask_vol[idx]
    
        # Fazendo o casting apropriado
        img = img.astype(np.float32)
        mask = mask.astype(np.int64)
    
        # Z-Score Normalization
        img = (img - img.mean()) / img.std()
    
        # Se for treino: crop aleatório
        if self.is_train:
            randh, randw = np.random.randint(0, 1024 - self.crop_size, size=2)
            img = img[randh:randh+self.crop_size, randw:randw+self.crop_size]
            mask = mask[randh:randh+self.crop_size, randw:randw+self.crop_size]
            
            img = np.stack([img, img, img], axis=0)  # (3, 256, 256)
            mask = np.expand_dims(mask, axis=0)  # (1, 256, 256)
        
        else:  # Validação: dividir em 16 partes e retornar cada uma separadamente
            patches = [
                (img[i:i+256, j:j+256], mask[i:i+256, j:j+256])
                for i in range(0, 1024, 256) for j in range(0, 1024, 256)
            ]
        
            img, mask = patches[idx % 16]  # Cada chamada retorna um patch individual
            
            img = np.stack([img, img, img], axis=0)  # (3, 256, 256)
            mask = np.expand_dims(mask, axis=0)  # (1, 256, 256)
    
        # Convertendo para tensor
        img = torch.tensor(img, dtype=torch.float32)
        mask = torch.tensor(mask, dtype=torch.int64)
    
        if self.transform:
            img = self.transform(img)    # (3, 224, 224)
            mask = self.transform(mask)  # (1, 224, 224)
    
        return img, mask

### Datasets e Dataloaders
#### 1) Crie os Dataloaders


In [ ]:
# Define a transformação com Compose
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensiona para o tamanho desejado
])

# 1) train_dataset =
# 2) valid_dataset =

print('Número de instâncias de treino:', len(train_dataset))
print('Número de instâncias de validação:', len(valid_dataset))


# 1) train_dataloader = -> batch_size=32, shuffle=True, num_workers=4
# 2) valid_dataloader = -> batch_size=32, shuffle=False, num_workers=4



# Verificando tamanhos e dados esperados
for imgs, masks in train_dataloader:
    print(imgs.shape)  # Esperado: torch.Size([32, 3, 224, 224])
    print(masks.shape)  # Esperado: torch.Size([32, 1, 224, 224])
    print(imgs.dtype, masks.dtype) # Esperado: torch.float32 torch.int64
    break
# Exemplo de iteração para validação
for imgs, masks in val_dataloader:
    print(imgs.shape)  # Esperado: torch.Size([32, 3, 224, 224])
    print(masks.shape)  # Esperado: torch.Size([32, 1, 224, 224])
    print(imgs.dtype, masks.dtype) # Esperado: torch.float32 torch.int64
    break

### Exemplo de Dados

In [ ]:
for imgs, masks in train_dataloader:
    print("Exibindo imagens do batch:")
    
    for i in range(1):
        show_image(imgs[i], masks[i])  # Mostrando a imagem i do batch
    break  

## Definindo o Modelo - ViT Segmentation

Aqui iremos definir a classe do nosso modelo `ViTSegmentation`. Para isso, utilizaremos um ViT pré-treinado (`vit_model`) como *encoder* e, em seguida, aplicaremos um *decoder* (`head`), que consistirá em uma camada `Conv2D` simples para realizar a segmentação. Após essa etapa, faremos um *upsample* para que a saída do modelo tenha o mesmo tamanho da máscara (`mask`).
#### 2) Defina ViTSegmentation

In [ ]:
class ViTSegmentation(nn.Module):
    def __init__(self, vit_model, num_classes):
        super(ViTSegmentation, self).__init__()

        # 1) Defina self.vit
        # 2) Defina self.decoder -> 768 é a dimensão do embedding do ViT
        # 3) defina self.upsample -> fator 16x


    def forward(self, x):

        # 1) Passe o input pelo ViT
        # 2) Pegue o último estado com .last_hidden_state -> Formato: (batch_size, num_patches + 1, 768)
        # 3) Tire o CLS (não iremos classificar, mas ele vem de padrão no modelo pré-trinado)
        # 4) Defina algo para patches.shape
        # 5) Defina patch_size = sqrt(num_patches)
        # 6) Redimensionar para (batch_size, embedding_dim, patch_size, patch_size)
        # 7) Passe pelo Decoder
        # 8) Faça o Upsample

  
        return outputs

#### Carregando o modelo pré-treinado
> É normal que dê algum Warning de peso não inicializado

In [ ]:
# Carregar ViT pré-treinado
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224")
model = ViTSegmentation(vit_model, num_classes=2)

### Testando plots e tamanhos passados no modelo

In [ ]:
for images, masks in val_dataloader:
    print(f"Batch de imagens: {images.shape}")
    print(f"Batch de máscaras: {masks.shape}")
    outputs = model(images)
    print(f"Saída do modelo: {outputs.shape}")

    show_image_and_mask_prediction(images[31], masks[31], outputs[31])
    break

# Resultado Esperado
# Batch de imagens: torch.Size([32, 3, 224, 224])
# Batch de máscaras: torch.Size([32, 1, 224, 224])
# Saída do modelo: torch.Size([32, 2, 224, 224])

## Treinamento e Avaliação (Exemplo básico)

#### 3) Escolha o otimizador e loss mais adequados

In [ ]:
weight_decay = 5e-4
learning_rate = 1e-4

# Escolha o otimizador e loss mais adequados
# 1) optimizer =
# 2) criterion =


#### 4) Faça o loop de treino

In [ ]:
def train(model, train_dataloader, criterion, optimizer, device):
    tic = time.time()
    
    model.to(device)
    model.train()
    train_losses = []
    all_labels, all_preds = [], []
    
    for i, batch in (pbar := tqdm(enumerate(train_dataloader), total=len(train_dataloader), unit='batch')):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.squeeze(1).long()

        # Faça o loop de treino padrão
        # .
        # .
        # .
        
        preds = outputs.argmax(dim=1)
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())
        
        train_losses.append(loss.item())
        pbar.set_description(f"Train loss: {np.mean(train_losses):.4f}")
    
    f1, iou = evaluate(all_preds, all_labels)
    tac = time.time()
    
    print('[train], [loss %.4f +/- %.4f], [iou %.4f +/- %.4f], [f1 %.4f +/- %.4f], [time %.2f]' % (
        np.mean(train_losses), np.std(train_losses), iou.mean(), iou.std(), f1.mean(), f1.std(), (tac - tic)))

def validate(model, valid_dataloader, criterion, epoch, device):
    tic = time.time()
    display_images_freq = 2
    
    model.to(device)
    model.eval()
    valid_losses = []
    all_labels, all_preds = [], []
    
    with torch.no_grad():
        for i, batch in (pbar := tqdm(enumerate(valid_dataloader), total=len(valid_dataloader), unit='batch')):
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.squeeze(1).long()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            preds = outputs.argmax(dim=1)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            
            valid_losses.append(loss.item())
            pbar.set_description(f"Valid loss: {np.mean(valid_losses):.4f}")
            
            if i == 0 and epoch % display_images_freq == 0:
                fig, ax = plt.subplots(2, 3, figsize=(10, 7))
                perm = np.random.permutation(inputs.size(0))
                
                for p in range(2):
                    ax[p, 0].imshow(inputs[perm[p], 0].cpu().numpy())
                    ax[p, 0].set_title('Image')
                    ax[p, 1].imshow(labels[perm[p]].cpu().numpy())
                    ax[p, 1].set_title('Label')
                    ax[p, 2].imshow(preds[perm[p]].cpu().numpy())
                    ax[p, 2].set_title('Prediction')
                    
                    for j in range(3):
                        ax[p, j].set_xticks([])
                        ax[p, j].set_yticks([])
                
                plt.show()
    
    f1, iou = evaluate(all_preds, all_labels)
    tac = time.time()
    
    print('[test], [loss %.4f +/- %.4f], [iou %.4f +/- %.4f], [f1 %.4f +/- %.4f], [time %.2f]' % (
        np.mean(valid_losses), np.std(valid_losses), iou.mean(), iou.std(), f1.mean(), f1.std(), (tac - tic)))


In [ ]:
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


for epoch in range(1, num_epochs + 1):
    print(f' ========== Epoch {epoch} ========== ')

    train(model, train_dataloader, criterion, optimizer, device)
    validate(model, val_dataloader, criterion, epoch, device)

## Conclusões

Assim como no primeiro notebook sobre ViT, essa mesma questão persiste: esse tipo de arquitetura precisa de muitos dados e tempo de treinamento para obter um bom desempenho nas tarefas que lhe forem atribuídas.

Dessa forma, o modelo levará muito mais tempo para convergir. Mesmo utilizando um modelo pré-treinado, ele foi ajustado para problemas de classificação, o que dificulta a adaptação para uma nova tarefa. Além disso, nossa cabeça de segmentação é extremamente simples, consistindo apenas em uma única convolução. Um cabeçote mais elaborado poderia melhorar os resultados das previsões.

Ainda assim, conseguimos métricas razoáveis e previsões coerentes, embora inferiores às obtidas com a UNet padrão, que alcançou bons resultados de forma muito mais rápida.

## **Desafio**
> Faça o ViT *from scratch*, defina TransformerLayer, TransformerEncoder, e uma SegmentationHead, gerando um modelo completo de segmentação que seria treinado do 0! Veja como ficam os resultados! (É normal que ele não fique bom, já que são modelos que demoram muito para treinar e *"fine-tunar"*).

> Se quiser, também pode buscar outros modelos, pré-treinados, ou usar o mesmo ViT model que fizemos acima e só usar uma SegmentationHead melhor!

In [ ]:
# Sua entrada tem o seguinte formato torch.Size([32, 3, 224, 224])

In [ ]:
def img_to_patch(x, patch_size):
    B, C, H, W = x.shape
    x = x.reshape(B, C, H // patch_size, patch_size, W // patch_size, patch_size)
    x = x.permute(0, 2, 4, 1, 3, 5)
    x = x.flatten(1,2)
    x = x.flatten(2,4)
    
    return x

In [ ]:
for img, label in train_dataloader: 
    print(img.shape, label.shape)  # Esperado = torch.Size([32, 3, 224, 224]) torch.Size([32, 1, 224, 224])
    x = img_to_patch(img, 16)      
    print(x.shape)                 # Esperado = torch.Size([32, 196, 768])
    break

In [ ]:
# Transformer Layer

class TransformerLayer(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_heads, dropout=0.0):
        super().__init__()

        
    def forward(self, x):
        
        return x

In [ ]:
x = torch.rand(32, 196, 768)

transformer_layer = TransformerLayer(embed_dim=768, hidden_dim=384, num_heads=8, dropout=0.1)
output = transformer_layer(x)

print(output.shape) # Esperado: torch.Size([32, 196, 768])

In [ ]:
class SegmentationHead(nn.Module):
    def __init__(self, in_channels=768, num_classes=1):
        super(SegmentationHead, self).__init__()
        
    def forward(self, x):
        
        return x


In [ ]:
x = torch.rand(32, 768, 14, 14)

segmentation_head = SegmentationHead()
output = segmentation_head(x)

print(output.shape)  # Esperado = torch.Size([32, 1, 224, 224])

In [ ]:
class Vit_Seg(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_channels, num_heads, num_layers, num_classes, patch_size, num_patches, dropout=0.0):
        super().__init__()

        # ...
        self.transformer = ...
        self.segmentation_head = ...
        # ...     
       

    
    def forward(self, x):
        
        # ...
        x = self.transformer(x)
        B, N, C = x.shape
        # ...
        x = self.segmentation_head(x) # Espera uma entrada de tamanhos [(32, 768, 14, 14)]
        # ...

        return x

In [ ]:
x = torch.rand(32, 3, 224, 224)

model = Vit_Seg(embed_dim=768, hidden_dim=384, num_channels=3, num_heads=8, num_layers=4, num_classes=2, patch_size=16, num_patches=196, dropout=0.1)
output = model(x)

print(output.shape)  # Esperado = torch.Size([32, 1, 224, 224])

In [ ]:
def train(model, train_dataloader, criterion, optimizer, device):
    tic = time.time()
    
    model.to(device)
    model.train()
    train_losses = []
    all_labels, all_preds = [], []
    
    for i, batch in (pbar := tqdm(enumerate(train_dataloader), total=len(train_dataloader), unit='batch')):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        # labels = labels.squeeze(1).long() -> p/ CrossEntropy
        labels = labels.float() # -> p/ BCELoss

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        preds = outputs.argmax(dim=1)
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())
        
        train_losses.append(loss.item())
        pbar.set_description(f"Train loss: {np.mean(train_losses):.4f}")

def validate(model, valid_dataloader, criterion, epoch, device):
    tic = time.time()
    display_metrics_freq = 5
    
    model.to(device)
    model.eval()
    valid_losses = []
    all_labels, all_preds = [], []
    
    with torch.no_grad():
        for i, batch in (pbar := tqdm(enumerate(valid_dataloader), total=len(valid_dataloader), unit='batch')):
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            # labels = labels.squeeze(1).long() -> p/ CrossEntropy
            labels = labels.float() # -> p/ BCELoss
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            preds = outputs.argmax(dim=1)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            
            valid_losses.append(loss.item())
            pbar.set_description(f"Valid loss: {np.mean(valid_losses):.4f}")
            
            if i == 0 and epoch % display_metrics_freq == 0:
                f1, iou = evaluate(all_preds, all_labels)
                tac = time.time()
    
                print('[test], [loss %.4f +/- %.4f], [iou %.4f +/- %.4f], [f1 %.4f +/- %.4f], [time %.2f]' % (
                    np.mean(valid_losses), np.std(valid_losses), iou.mean(), iou.std(), f1.mean(), f1.std(), (tac - tic)))


In [ ]:
weight_decay = 5e-4
learning_rate = 1e-4

# Escolha o otimizador e loss mais adequados -> Dica BCELoss ou CrossEntropy
# 1) optimizer =
# 2) criterion =

In [ ]:
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss()

for epoch in range(1, num_epochs + 1):
    print(f' ========== Epoch {epoch} ========== ')

    train(model, train_dataloader, criterion, optimizer, device)
    validate(model, val_dataloader, criterion, epoch, device)

In [ ]:
# Teste mais coisas !

In [ ]:
# ...